In [110]:
import pandas as pd
from config import config

from preprocessors import get_meta_columns


ImportError: cannot import name 'get_meta_columns' from 'preprocessors' (/Users/matthewgalloway/Repos/dataiku_interview/preprocessors.py)

In [111]:
data = pd.read_csv(config.TRAIN_DIR, names=get_meta_columns())
data.head()

AttributeError: module 'config' has no attribute 'META_DIR'